In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import cv2
import numpy as np

 
def display_image(image_path: str):
    # Load the 16-bit image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    print(image.dtype, image.max())
    # Reverse gamma correction
    gamma = 2.2
    linearized_image = (image / image.max()) ** (1 / gamma)
    print(linearized_image.dtype, linearized_image.max())
    # Scale to 16-bit range
    #scaled_image = (linearized_image * image.max()).astype(np.uint16)

    cv2.imwrite("test.jpeg", (linearized_image * 65535).astype(np.uint16))
    cv2.imshow('Image', (linearized_image * 65535).astype(np.uint16))

    while True:
        key = cv2.waitKey(1) & 0xFF 
        if key == ord('q'):
            break
    cv2.destroyWindow('Image')
    cv2.waitKey(1)


In [2]:
display_image("Data/Canon_5d_1/png/IMG_0411.png")

uint16 3692
float64 1.0


In [2]:
import scipy.io

mat_data = scipy.io.loadmat('assests/groundtruth_568/colourchecker_gamma1_bit12.mat')
for key in mat_data:
    # Skip keys that are a part of MATLAB's metadata
    if key.startswith('__'):
        continue

    data = mat_data[key]
    print(data.shape)

(568, 24, 3)


# Understanding the working of Data_preparation steps

In [1]:
import configparser
from Data_preparation import get_shigehler_patch_corners

def main(shigehler_config):
    shigehler_patch_corners = get_shigehler_patch_corners(shigehler_config)
if __name__ == "__main__":
    shigehler_config = configparser.ConfigParser()
    shigehler_config.read("assests/shigehler.cfg")
    
    main(shigehler_config)

Getting Shigehler patch corners....
568


In [1]:
import configparser
import numpy as np

In [2]:
dataset = np.load("Data/shigehler.npz")
patches = dataset['data']
patch_labels = dataset['patch_labels']
image_idx = dataset['image_idx']

In [3]:
print("patches shape: {}".format(patches.shape))
print("patch_labels shape: {}".format(patch_labels.shape))
print("image_idx shape: {}".format(image_idx.shape))

patches shape: (56800, 32, 32, 3)
patch_labels shape: (56800, 3)
image_idx shape: (56800,)


In [4]:
import torch
# Convert numpy arrays to torch tensors.
patches_tensor = torch.from_numpy(patches).permute(0, 3, 1, 2).float()
patches_labels_tensor = torch.from_numpy(patch_labels).float()

# Split the dataset into train and val.
num_images = 568
patches_per_image = 100
train_images = int(0.8 * num_images)
train_size = train_images * patches_per_image

train_patches, train_patch_labels, train_idx = patches_tensor[:train_size], patches_labels_tensor[:train_size], image_idx[:train_size]
val_patches, val_patch_labels, val_idx = patches_tensor[train_size:], patches_labels_tensor[train_size:], image_idx[train_size:]

In [5]:
print(f"{train_patches.shape}, {train_patch_labels.shape}, {train_idx.shape}")
print(f"{val_patches.shape}, {val_patch_labels.shape}, {val_idx.shape}")

torch.Size([45400, 3, 32, 32]), torch.Size([45400, 3]), (45400,)
torch.Size([11400, 3, 32, 32]), torch.Size([11400, 3]), (11400,)


In [6]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(train_patches, train_patch_labels)
val_dataset = TensorDataset(val_patches, val_patch_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

print(f"{train_loader}, {val_loader}")

<torch.utils.data.dataloader.DataLoader object at 0x12780ad90>, <torch.utils.data.dataloader.DataLoader object at 0x12780a070>


In [7]:
from Models import IlluminantEstimationCNN
from train import euclidean_distance_loss
import torch.optim as optim

model = IlluminantEstimationCNN()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)

device = "mps"
model = IlluminantEstimationCNN().to(device)
# Training loop with validation.
num_epochs = 2
best_loss = float('inf')
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = euclidean_distance_loss(target, output)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = euclidean_distance_loss(target, output)
            total_val_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)
    
    # Print statistics
    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")

    # Save the best model
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_model.pth')
    
print("Training Finished!")

Epoch [1/2], Train Loss: 1.2322411508607225, Val Loss: 1.2303143743039513
Epoch [2/2], Train Loss: 1.2322375285449039, Val Loss: 1.2303143743039513
Training Finished!


In [ ]:
model.eval()  # set model to evaluation mode
predictions_dict = {}

with torch.no_grad():
    for image, label, idx in zip(patches_tensor, patch_labels_tensor, image_idx):
        output = model(image.unsqueeze(0))  # Add batch dimension
        # Convert tensor output to numpy and remove batch dimension
        pred = output.squeeze(0).numpy()
        
        if idx in predictions_dict:
            predictions_dict[idx].append(pred)
        else:
            predictions_dict[idx] = [pred]
